## Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости.

Заказчику важны:
  - качество предсказания;
  - скорость предсказания;
  - время обучения.

### Описание проекта

Необходимо обучить разные модели. Для каждой попробовать различные гиперпараметры. <br>Проанализируйте скорость работы и качество моделей.<br>
Примечания:<br>
Для оценки качества моделей нужно применить метрику RMSE.<br>


#### 1. Загрузка и подготовка данных

Импрртируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
warnings.filterwarnings("ignore")

Загружаем данные

In [2]:
import os

pth1 = 'autos.csv'
pth2 = '/datasets/autos.csv'

if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print('Something is wrong')

изучаем размер датасета и информацию по нему

In [3]:
df.shape

(354369, 16)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
df.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Переводим названия столбцов в нижний регистр

In [6]:
df.columns = df.columns.str.lower()

заполняем пропуски - 'unknown'

In [7]:
df = df.fillna('unknown')

Проверяем

In [8]:
df.head(5)

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
0,2016-03-24 11:52:17,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


проверяем наличие дубликатов

In [9]:
df.duplicated().sum()

4

удаляем дубликаты

In [10]:
df = df.drop_duplicates()

Проверем

In [11]:
df.duplicated().sum()

0

Изучаем уникальные значения в датасете

In [12]:
df['registrationyear'].unique()

array([1993, 2011, 2004, 2001, 2008, 1995, 1980, 2014, 1998, 2005, 1910,
       2016, 2007, 2009, 2002, 2018, 1997, 1990, 2017, 1981, 2003, 1994,
       1991, 1984, 2006, 1999, 2012, 2010, 2000, 1992, 2013, 1996, 1985,
       1989, 2015, 1982, 1976, 1983, 1973, 1111, 1969, 1971, 1987, 1986,
       1988, 1970, 1965, 1945, 1925, 1974, 1979, 1955, 1978, 1972, 1968,
       1977, 1961, 1960, 1966, 1975, 1963, 1964, 5000, 1954, 1958, 1967,
       1959, 9999, 1956, 3200, 1000, 1941, 8888, 1500, 2200, 4100, 1962,
       1929, 1957, 1940, 3000, 2066, 1949, 2019, 1937, 1951, 1800, 1953,
       1234, 8000, 5300, 9000, 2900, 6000, 5900, 5911, 1933, 1400, 1950,
       4000, 1948, 1952, 1200, 8500, 1932, 1255, 3700, 3800, 4800, 1942,
       7000, 1935, 1936, 6500, 1923, 2290, 2500, 1930, 1001, 9450, 1944,
       1943, 1934, 1938, 1688, 2800, 1253, 1928, 1919, 5555, 5600, 1600,
       2222, 1039, 9996, 1300, 8455, 1931, 1915, 4500, 1920, 1602, 7800,
       9229, 1947, 1927, 7100, 8200, 1946, 7500, 35

Видно что имеются не корректные года

In [13]:
df['brand'].unique()

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'opel', 'seat',
       'citroen', 'honda', 'fiat', 'mini', 'smart', 'hyundai',
       'sonstige_autos', 'alfa_romeo', 'subaru', 'volvo', 'mitsubishi',
       'kia', 'suzuki', 'lancia', 'toyota', 'chevrolet', 'dacia',
       'daihatsu', 'trabant', 'saab', 'chrysler', 'jaguar', 'daewoo',
       'porsche', 'rover', 'land_rover', 'lada'], dtype=object)

In [14]:
df['registrationmonth'].unique()

array([ 0,  5,  8,  6,  7, 10, 12, 11,  2,  3,  1,  4,  9], dtype=int64)

In [15]:
df['numberofpictures'].unique()

array([0], dtype=int64)

In [16]:
df['kilometer'].unique()

array([150000, 125000,  90000,  40000,  30000,  70000,   5000, 100000,
        60000,  20000,  80000,  50000,  10000], dtype=int64)

In [17]:
df['vehicletype'].unique()

array(['unknown', 'coupe', 'suv', 'small', 'sedan', 'convertible', 'bus',
       'wagon', 'other'], dtype=object)

In [18]:
df['gearbox'].unique()

array(['manual', 'auto', 'unknown'], dtype=object)

In [19]:
df['model'].unique()

array(['golf', 'unknown', 'grand', 'fabia', '3er', '2_reihe', 'other',
       'c_max', '3_reihe', 'passat', 'navara', 'ka', 'polo', 'twingo',
       'a_klasse', 'scirocco', '5er', 'meriva', 'arosa', 'c4', 'civic',
       'transporter', 'punto', 'e_klasse', 'clio', 'kadett', 'kangoo',
       'corsa', 'one', 'fortwo', '1er', 'b_klasse', 'signum', 'astra',
       'a8', 'jetta', 'fiesta', 'c_klasse', 'micra', 'vito', 'sprinter',
       '156', 'escort', 'forester', 'xc_reihe', 'scenic', 'a4', 'a1',
       'insignia', 'combo', 'focus', 'tt', 'a6', 'jazz', 'omega', 'slk',
       '7er', '80', '147', '100', 'z_reihe', 'sportage', 'sorento', 'v40',
       'ibiza', 'mustang', 'eos', 'touran', 'getz', 'a3', 'almera',
       'megane', 'lupo', 'r19', 'zafira', 'caddy', 'mondeo', 'cordoba',
       'colt', 'impreza', 'vectra', 'berlingo', 'tiguan', 'i_reihe',
       'espace', 'sharan', '6_reihe', 'panda', 'up', 'seicento', 'ceed',
       '5_reihe', 'yeti', 'octavia', 'mii', 'rx_reihe', '6er', 'modus',

In [20]:
df['fueltype'].unique()

array(['petrol', 'gasoline', 'unknown', 'lpg', 'other', 'hybrid', 'cng',
       'electric'], dtype=object)

В столбце - Год регистрации авто - попадаются некорректные года. Это может бытьсвязано как с ошибками автовладельцев заполнявшими данными, так и с ошибками возникшими при выгрузке.

Можно предположить то авто выставленные на продажу скорее всего использовались в повседневной жизни и скорее всего машина не могла ездить меньше 5 000 км в  год, тк максимальный пробег авто составляет - 150 000 км.  По этому можно сделать вывод что автомобиль не может быть старше 30 лет в пригодном состоянии. К тому же вряд ли найдется авто старше 30 лет который будет продаваться не на специализированных плащадках для ретро автомобилей.

In [21]:
display(df[df['registrationyear'] == 2900])

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
46741,2016-03-12 07:36:23,5000,unknown,2900,manual,120,corsa,5000,9,petrol,opel,unknown,2016-03-12 00:00:00,0,9326,2016-04-05 23:18:26


Делаем срез датасета по авто у которых стоит год регистрации больше 2016

In [22]:
over_2016 = df.query('registrationyear > 2016')
over_2016.head(5)


,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
22,2016-03-23 14:52:51,2900,unknown,2018,manual,90,meriva,150000,5,petrol,opel,no,2016-03-23 00:00:00,0,49716,2016-03-31 01:16:33
26,2016-03-10 19:38:18,5555,unknown,2017,manual,125,c4,125000,4,unknown,citroen,no,2016-03-10 00:00:00,0,31139,2016-03-16 09:16:46
48,2016-03-25 14:40:12,7750,unknown,2017,manual,80,golf,100000,1,petrol,volkswagen,unknown,2016-03-25 00:00:00,0,48499,2016-03-31 21:47:44
51,2016-03-07 18:57:08,2000,unknown,2017,manual,90,punto,150000,11,gasoline,fiat,yes,2016-03-07 00:00:00,0,66115,2016-03-07 18:57:08
57,2016-03-10 20:53:19,2399,unknown,2018,manual,64,other,125000,3,unknown,seat,no,2016-03-10 00:00:00,0,33397,2016-03-25 10:17:37


In [23]:
over_2016.shape

(14530, 16)

создаем список годов которыми мы будем заполнять те ячейки в которых года больше 2016

In [24]:
import random
list_over_2016 = []
for i in range(2010,2017):
     list_over_2016.append(i)
print(list_over_2016)


[2010, 2011, 2012, 2013, 2014, 2015, 2016]


при помощи лямбда функции заполняем столбец 'registrationyear' по условию

In [25]:
df['registrationyear'] = df['registrationyear'].map(lambda x: np.random.choice(list_over_2016) if x > 2016 else x)

Делаем срез датасета по авто у которых стоит год регистрации меньше 1990

In [26]:
less_1990 = df.query('registrationyear < 1990')
less_1990.head(5)


,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
15,2016-03-11 21:39:15,450,small,1910,unknown,0,ka,5000,0,petrol,ford,unknown,2016-03-11 00:00:00,0,24148,2016-03-19 08:46:47
28,2016-03-09 11:56:38,1990,bus,1981,manual,50,transporter,5000,1,petrol,volkswagen,no,2016-03-09 00:00:00,0,87471,2016-03-10 07:44:33
39,2016-03-25 15:50:30,1500,bus,1984,manual,70,other,150000,8,gasoline,mercedes_benz,no,2016-03-25 00:00:00,0,22767,2016-03-27 03:17:02
91,2016-04-02 23:25:25,1222,sedan,1984,auto,180,other,150000,3,petrol,opel,unknown,2016-04-02 00:00:00,0,57290,2016-04-06 14:44:57


создаем список годов в интервале 1986 до 1990

In [27]:
import random
list_less_1990 = []
for i in range(1986,1991):
     list_less_1990.append(i)
print(list_less_1990)


[1986, 1987, 1988, 1989, 1990]


при помощи лямбда функции заполняем столбец 'registrationyear' по условию

In [28]:
df['registrationyear'] = df['registrationyear'].map(lambda x: np.random.choice(list_less_1990) if x<1990 else x)

Проверем

In [29]:
display(df.query('1990 < registrationyear > 2016'))

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen


In [30]:
df.shape

(354365, 16)

In [31]:
df.head(5)

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen
0,2016-03-24 11:52:17,480,unknown,1993,manual,0,golf,150000,0,petrol,volkswagen,unknown,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,unknown,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,unknown,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


меняем значение на 1, тк нам не принципиален месяц регистрации. Он не влияетна стоимость авто

удаляем столбцы: datecrawled, datecreated, lastseen, numberofpictures тк эти столбцы не нужны для построения модели предсказания стоимости автомобиля

In [32]:
df = df.drop(['datecrawled', 'datecreated', 'numberofpictures', 'lastseen', 'postalcode','registrationmonth'], axis=1)

In [33]:
df = df.iloc[1:]

In [34]:
df.head(5)

,price,vehicletype,registrationyear,gearbox,power,model,kilometer,fueltype,brand,notrepaired
1,18300,coupe,2011,manual,190,unknown,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,unknown
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes


In [35]:
print(df[df['power']==0]['price'].count())

40224


In [36]:
df = df.loc[df['power']!=0]

In [37]:
print(df[df['power']==0]['price'].count())

0


In [38]:
print(df[df['price']==0]['power'].count())

6907


In [39]:
df = df.loc[df['price']!=0]

In [40]:
print(df[df['price']==0]['power'].count())

0


#### Вывод 

Загрузили данные.<br>
Перевели названия столбцов в нижний регистр. <br>
Заполнили пропуски - 'unknown'<br>
Проверили наличие дубликатов - 4 шт <br>
Удалили дубликаты<br>
Изучили уникальные значения в датасете<

В столбце - Год регистрации авто - попадаются некорректные года. Это может бытьсвязано как с ошибками автовладельцев заполнявшими данными, так и с ошибками возникшими при выгрузке.

Предположили что автомобиль не может быть старше 30 лет в пригодном состоянии. 

По этому сделали срез датасета по авто у которых стоит год регистрации больше 2016 и меньше 1990
При помощи лямбда функции заполнили столбец 'registrationyear' по условию

Удалили столбцы: datecrawled, datecreated, lastseen, numberofpictures тк эти столбцы не нужны для построения модели предсказания стоимости автомобиля

Для построения моделей перевели все предметные признаки в числовые при помощи  OrdinalEncoder

#### 2. Обучение моделей

Кодирование OrdinalEncoder

In [41]:
categories = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired']
df_oe = df.copy()
encoder = OrdinalEncoder()
encoder.fit(df[categories])
df_oe[categories] = encoder.transform(df_oe[categories])

features_oe = df_oe.drop('price', axis=1)
target_oe = df_oe['price']
features_train_oe, features_test_oe, target_train_oe, target_test_oe = train_test_split(
    features_oe, target_oe, test_size=0.3, random_state=123)



кодирование OHE

In [42]:
cat_col = ['vehicletype', 'gearbox','model', 'fueltype', 'brand', 'notrepaired']
df[cat_col] = df[cat_col].astype('category')

features_categorial = pd.get_dummies(df, drop_first=True)


features_cat = features_categorial.drop('price', axis=1)
target_cat = features_categorial['price']
features_train_cat, features_test_cat, target_train_cat, target_test_cat = train_test_split(
    features_cat, target_cat, test_size=0.3, random_state=123)

Выделяем: признаки и целевой признак

In [43]:
target = df['price']
features = df.drop('price', axis=1)


Разбиваем выборку на обучающую и валидную 

In [44]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.30, random_state=12345)

In [45]:
print('Размер обучающей выборки {}, от общего количества данных {:.1%}'.format(
    len(features_train), len(features_train)/features.shape[0]))

print('Размер тестовой выборки {}, от общего количества данных {:.1%}'.format(
    len(features_test), len(features_test)/features.shape[0]))

Размер обучающей выборки 215063, от общего количества данных 70.0%
Размер тестовой выборки 92170, от общего количества данных 30.0%


In [46]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

<b>Дерево решений</b>

создаем функцию дя обучения модели Дерево Решений. Используем кросс-валидацию, в качестве оценки используем RMSE

In [47]:
def tree_decision(features, target):
        
    for depth in range(1, 10):
        model = DecisionTreeRegressor(random_state=12345, max_depth=depth) 
        score = cross_val_score(model, features, target, cv=cv, scoring='neg_mean_absolute_error').mean()
        
                    
    return score
                    
    

Время работы модели:

In [48]:
%%time
result_tree_decision = tree_decision(features_train_oe, target_train_oe)

Wall time: 1min 36s


In [49]:
print('Знаение RMSE Дерева Решений - ', round(abs(result_tree_decision), 2))

Знаение RMSE Дерева Решений -  1378.22


<b>Линейная Регрессия</b>

создаем функцию дя обучения модели Линейной Регрессии. Используем кросс-валидацию, в качестве оценки используем RMSE

In [50]:
def regression(features, target):

    model = LinearRegression()
    score = cross_val_score(model, features, target, cv=cv, scoring='neg_mean_absolute_error').mean()
    return score

Врем работы модели:

In [51]:
%%time
result = regression(features_train_cat, target_train_cat)


Wall time: 2min 40s


In [52]:
print('Знаение RMSE Линейной Регрессии - ', round(abs(result),2))

Знаение RMSE Линейной Регрессии -  1865.93


Модель Случайного леса

In [53]:
def random_forest(features, target):
    
       
    for estimators in range(1, 5):
        for depth in range(1, 10):
            model = RandomForestRegressor(random_state=12345, n_estimators=estimators,  max_depth=depth)
            score = cross_val_score(model, features, target, cv=cv, scoring='neg_mean_absolute_error').mean()
                        
    return  score    

Время работы модели:

In [54]:
%%time
result_rf = random_forest(features_train_oe, target_train_oe)


Wall time: 11min 15s


In [55]:
print('Знаение RMSE Случайного леса -', round(abs(result_rf), 2))

Знаение RMSE Случайного леса - 1334.93


<b>Проверяем модель которая показала себя луше всего - Случайный лес</b>

In [69]:
%%time
result_rf_test = random_forest(features_test_oe, target_test_oe)

Wall time: 4min 23s


In [70]:
print('Знаение RMSE модели Случайного леса на валидных данных -', round(abs(result_rf_test), 2))

Знаение RMSE модели Случайного леса на валидных данных - 1340.6


Использование библиотеки LightGBM

In [56]:
def lgbmregress(features, target):
    model = LGBMRegressor(boosting_type='gbdt', categorial_feature = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand', 'notrepaired'],num_leaves=31, max_depth=- 1, learning_rate=0.1, n_estimators=100)
    score = cross_val_score(model, features, target, cv=cv, scoring='neg_mean_absolute_error', n_jobs=-1, error_score='raise').mean()
    return score

Время работы модели:

In [57]:
%%time
result_lgbmr = lgbmregress(features_train, target_train)

Wall time: 36.4 s


In [58]:
print('Знаение RMSE модели с использованием библиотеки LightGBM -', round(abs(result_lgbmr),2))

Знаение RMSE модели с использованием библиотеки LightGBM - 1049.02


Проверяем модель с использованием библиотеки LightGBM и тестируем модель

In [72]:
%%time
result_lgbmr = lgbmregress(features_test, target_test)
print('Знаение RMSE модели с использованием библиотеки LightGBM -', round(abs(result_lgbmr),2))

Знаение RMSE модели с использованием библиотеки LightGBM - 1066.31
Wall time: 36.6 s


#### Вывод

Обучили модели на тренировочных данных.<br>
В качестве функции потерь мы используем показатель RMSE. <br>
Быстрее всего обучается Модель Дерево решений   - около полутора минут, но при этом выдает не лучший результат - 1378.22 <br>
Модель Линейная регрессия обучается медленее - приблизительно 3  минуты, но дает хучший результат  - 1865.93  <br>
Модель Случайный лес обучается дольше всего - порядка 11 минут, при том выдает лучший лучший результат - 1334.93

Использование библиотеки LightGBM дает лучший результат - 1049.02 при небольшом затраченом времени на обучение - около 40 секунд .

Тестируем лучшую модель и используем библиотеку LightGBM<br>
Случайный лес дает хороший результат на тестовых данных - 1340.6, но при этом обучается  долго - 4 с половиной минуты <br>
Модель с библиотекой LightGBM показывает лучший результат - 1066.31 при недольшом затраченном времени около 40 секунд. 

## Итоговый Вывод

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости.
Нам было необходимо было создать модель дя определения стоимости автомобилей.

Перед нами были поставлены следующие задачи для модели:

качество предсказания;
скорость предсказания;
время обучения.


   Обучил разные модели. Для каждой попробовал различные гиперпараметры.
Проанализировал скорость работы и качество моделей.

   Были импортированы все необходимые библиотеки и модули.<br>
Загружены и изучены данные.<br>
Привели названия столбцов в нижнийрегистр для удобства.<br>
Были найдены и удалены дубликаты. Были обнаружены пропуски и заменены на 'unknown'.<br>
Изучены уникальные значения датафрейма.

   В столбце - Год регистрации авто - попадаются некорректные года. Это может бытьсвязано как с ошибками автовладельцев заполнявшими данными, так и с ошибками возникшими при выгрузке.<br>
   Можно предположить то авто выставленные на продажу скорее всего использовались в повседневной жизни и скорее всего машина не могла ездить меньше 5 000 км в год, тк максимальный пробег авто составляет - 150 000 км. По этому можно сделать вывод что автомобиль не может быть старше 30 лет в пригодном состоянии. К тому же вряд ли найдется авто старше 30 лет который будет продаваться не на специализированных плащадках для ретро автомобилей.<br>
   По этому сделали срез датасета по авто у которых стоит год регистрации больше 2016 и меньше 1990 При помощи лямбда функции заполнили столбец 'registrationyear' по условию.

Удалили столбцы: datecrawled, datecreated, lastseen, numberofpictures тк эти столбцы не нужны для построения модели. предсказания стоимости автомобиля

Для построения моделей перевели все предметные признаки в числовые при помощи OrdinalEncoder.
 
 Выделили признаки и целевой признак

Обучили модели на тренировочных данных.<br>
В качестве функции потерь мы используем показатель RMSE. <br>
Быстрее всего обучается Модель Дерево решений   - около полутора минут, но при этом выдает не лучший результат - 1378.22 <br>
Модель Линейная регрессия обучается медленее - приблизительно 3  минуты, но дает хучший результат  - 1865.93  <br>
Модель Случайный лес обучается дольше всего - порядка 11 минут, при том выдает лучший лучший результат - 1334.93

Использование библиотеки LightGBM дает лучший результат - 1049.02 при небольшом затраченом времени на обучение - около 40 секунд .

Тестируем лучшую модель и используем библиотеку LightGBM<br>
Случайный лес дает хороший результат на тестовых данных - 1340.6, но при этом обучается  долго - 4 с половиной минуты <br>
Модель с библиотекой LightGBM показывает лучший результат - 1066.31 при недольшом затраченном времени около 40 секунд. 
   
В Итоге мы поучили рабочую модель которая минимизирует ошибки и дает высокую точность результатов